In [1]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path

In [2]:
# auxiliary function for below
def split(theList, n):
    for i in range(0, len(theList), n):
        yield theList[i:i + n]

In [3]:
host = "https://api.stratz.com/api/v1/match"

In [5]:
params = {"matchId":[3304258209,3304204784], 
          "include": ["Player"],
         "gameMode": [2, 22],
         "lobbyType": [2, 7]}

In [6]:
x = requests.get(host, params = params)

In [7]:
x

<Response [200]>

In [8]:
host = "https://api.stratz.com/api/v1/match/3304258209"
rs = (grequests.get(host),)
x = grequests.map(rs)
data = x[0].json()

In [8]:
keys = data.keys()
print(keys)

dict_keys(['clusterId', 'challenge', 'id', 'lobbyType', 'rank', 'roshanEvents', 'gameVersionId', 'runeEvents', 'isStats', 'gameResult', 'firstBloodTime', 'didRadiantWin', 'duration', 'gameMode', 'parsedDate', 'wardEvents', 'towerDeathEvents', 'avgImp', 'bracket', 'regionId', 'direKills', 'players', 'buildingEvents', 'radiantKills', 'replaySalt', 'tier', 'courierEvents', 'endDate'])


In [9]:
testsequence = [event['item'] for event in data['players'][0]['purchaseEvents']]
testitems = [45, 42, 44]
list(filter(lambda a: a not in testitems, testsequence))

[188,
 43,
 29,
 38,
 46,
 46,
 84,
 244,
 27,
 27,
 214,
 46,
 46,
 46,
 25,
 182,
 46,
 46,
 64,
 65,
 46,
 46,
 23,
 21,
 22,
 60,
 108]

In [10]:
keys

dict_keys(['clusterId', 'challenge', 'id', 'lobbyType', 'rank', 'roshanEvents', 'gameVersionId', 'runeEvents', 'isStats', 'gameResult', 'firstBloodTime', 'didRadiantWin', 'duration', 'gameMode', 'parsedDate', 'wardEvents', 'towerDeathEvents', 'avgImp', 'bracket', 'regionId', 'direKills', 'players', 'buildingEvents', 'radiantKills', 'replaySalt', 'tier', 'courierEvents', 'endDate'])

In [11]:
data['players'][0].keys()

dict_keys(['role', 'abilityLearnEvents', 'item3', 'heroDamage', 'level', 'abilityActiveLists', 'item4', 'item1', 'playerUpdateLevelEvents', 'item0', 'goldPerMinute', 'award', 'expPerMinute', 'healEvents', 'playerUpdateGoldEvents', 'soloRank', 'isRadiant', 'name', 'deathEvents', 'leaverStatus', 'lane', 'imp', 'playerUpdatePositionEvents', 'towerDamageEvents', 'inventoryEvent', 'assistEvents', 'heroDamageEvents', 'purchaseEvents', 'avatar', 'steamId', 'numDeaths', 'avatarMedium', 'numDenies', 'slot', 'killEvents', 'goldSpent', 'heroAvgImp', 'heroHealing', 'csEvents', 'avatarFull', 'abilityUsedEvents', 'item5', 'playerUpdateAttributeEvents', 'buyBackEvents', 'towerDamage', 'experienceEvents', 'numAssists', 'goldEvents', 'partyRank', 'hero', 'playerUpdateHealthEvents', 'profileUrl', 'numKills', 'playerUpdateBattleEvents', 'item2', 'numLastHits'])

In [12]:
print(data['players'][0]['role'])
print(data['players'][0]['lane'])
print(data['players'][0]['hero'])
print(data['players'][0]['purchaseEvents'])
for player in data['players']:
    print(player['slot'])

1
1
92
[{'item': 188, 'time': -89}, {'item': 43, 'time': -89}, {'item': 29, 'time': -89}, {'item': 44, 'time': -89}, {'item': 38, 'time': -89}, {'item': 46, 'time': -85}, {'item': 46, 'time': -84}, {'item': 84, 'time': 187}, {'item': 42, 'time': 193}, {'item': 42, 'time': 236}, {'item': 42, 'time': 384}, {'item': 244, 'time': 389}, {'item': 27, 'time': 389}, {'item': 27, 'time': 432}, {'item': 214, 'time': 440}, {'item': 46, 'time': 509}, {'item': 42, 'time': 528}, {'item': 46, 'time': 603}, {'item': 46, 'time': 719}, {'item': 42, 'time': 720}, {'item': 25, 'time': 809}, {'item': 182, 'time': 811}, {'item': 42, 'time': 816}, {'item': 46, 'time': 882}, {'item': 46, 'time': 1059}, {'item': 64, 'time': 1264}, {'item': 65, 'time': 1264}, {'item': 46, 'time': 1277}, {'item': 46, 'time': 1533}, {'item': 23, 'time': 1758}, {'item': 21, 'time': 1758}, {'item': 22, 'time': 1758}, {'item': 60, 'time': 1760}, {'item': 108, 'time': 1760}, {'item': 42, 'time': 1919}]
0
1
2
3
4
128
129
130
131
132


**Need to get steamIDs for players by going through the matchIDs of high level games**

In [13]:
def nextBatchOfMatches(idList):
    queryParams = {"matchId":idList, 
          "include": ["Player"],
         "gameMode": [2, 22],
         "lobbyType": [2, 7]}
    x = requests.get(host, params = queryParams)
    x = x.json()
    for entry in x['results']:
        for player in entry['players']:
            playerSet.add(player['steamId'])

In [14]:
if os.path.exists('playerset.txt'):
    playerSet = np.loadtxt('playerset.txt', dtype=np.int64)
else:
    playerSet = set()
    allMatches = np.loadtxt('matchids.txt', dtype=np.int64)
    allMatches = split(allMatches, 10)
    for matches in allMatches:
        nextBatchOfMatches(matches)
    playerSet = list(playerSet)
    np.savetxt('playerset.txt', playerSet, delimiter=',', fmt = "%.0f")

With player set, we can obtain matches. For the players above, get the matches these players play and also categorize the players and heroes

In [15]:
# 8 is jug and 74 is invoker and 106 is ember
def playerMatchObtain(player, heroList = [8, 74, 106]):
    host = "https://api.stratz.com/api/v1/match"
    queryParams = {"steamId": player, 
                   "heroId": heroList,
                   "gameMode": [2, 22],
                   "lobbyType": [2, 7],
                   "gameVersion": 79, 
                  "take": 100}
    x = requests.get(host, params = queryParams)
    x = x.json()
    for entry in x['results']:
        matchSet.add(entry['id'])

In [16]:
print('Loading matches')
matchSet = set()
if os.path.exists('matchset.txt'):
    matchSet = np.loadtxt('matchset.txt', dtype=np.int64)
else:
    allplayers = np.loadtxt('playerset.txt', dtype=np.int64)
    for player in allplayers:
        playerMatchObtain(player, heroList=[74])
    matchSet = list(matchSet)
    np.savetxt('matchset.txt', matchSet, delimiter=',', fmt = "%.0f")

In [17]:
len(playerSet)

589

In [18]:
len(matchSet)

1542

** Time to Parse these Matches**
Need to make sure to remove tps and wards from buy list as you can buy them at any time depending on the situation

In [35]:
prototypeDataframe = {
    "invokerItems": [], "invokerSideRadiant": [], 
    "invokerSideHero1": [], "invokerSideHero2": [], 
    "invokerSideHero3": [], "invokerSideHero4": [],
    "otherSideHero1": [], "otherSideHero2": [],
    "otherSideHero3": [], "otherSideHero4": [],
    "otherSideHero5": [], 
    "invokerSideHero1Steam": [], "invokerSideHero2Steam": [], 
    "invokerSideHero3Steam": [], "invokerSideHero4Steam": [],
    "otherSideHero1Steam": [], "otherSideHero2Steam": [],
    "otherSideHero3Steam": [], "otherSideHero4Steam": [],
    "otherSideHero5Steam": [], "matchId": []
}
session = requests.Session()
def analyzeMatchesForFeatures(matchList, heroesToTestItems = [8, 74, 106], ):
    hostInitial = "https://api.stratz.com/api/v1/match/"
    rs = (grequests.get(hostInitial + str(match), session=session) for match in matchList)
    x = grequests.map(rs, size = 50)
    for matchUnparsed in x:
        if not matchUnparsed.ok:
            matchUnparsed.close()
            continue
        match = matchUnparsed.json()
        radiantHeroes = []
        direHeroes = []
        radiantSteams = []
        direSteams = []
        invokerSideRadiant = None
        for player in match['players']:
            # we have the invoker
            if player['hero'] == 74:
                invokerSideRadiant = player['slot'] < 5
                invokerItems = [event['item'] for event in player['purchaseEvents']]
            else:
                if player['slot'] < 5:
                    radiantHeroes.append(player['hero'])
                    radiantSteams.append(player['steamId'])
                else: 
                    direHeroes.append(player['hero'])
                    direSteams.append(player['steamId'])
        invokerWon = invokerSideRadiant == match['didRadiantWin']
        if not invokerWon:
            matchUnparsed.close()
            continue
        prototypeDataframe['matchId'].append(match['id'])
        prototypeDataframe['invokerSideRadiant'].append(invokerSideRadiant)
        prototypeDataframe['invokerItems'].append(invokerItems)
        if invokerSideRadiant:
            invokerSideHeroes = radiantHeroes
            invokerSideSteams = radiantSteams
            otherSideHeroes = direHeroes
            otherSideSteams = direSteams
        else:
            invokerSideHeroes = direHeroes
            invokerSideSteams = direSteams
            otherSideHeroes = radiantHeroes
            otherSideSteams = radiantSteams
        for ix1 in range(5):
            prototypeDataframe['otherSideHero' + str(ix1+1)].append(otherSideHeroes[ix1])
            prototypeDataframe['otherSideHero' + str(ix1+1) + 'Steam'].append(otherSideSteams[ix1])
            if ix1 != 4:
                prototypeDataframe['invokerSideHero' + str(ix1+1)].append(invokerSideHeroes[ix1])
                prototypeDataframe['invokerSideHero'+ str(ix1+1) + 'Steam'].append(invokerSideSteams[ix1])
        matchUnparsed.close()

In [36]:
from scipy import stats
def GetHeroHistory(steamid, heroid):
    host = "https://api.stratz.com/api/v1/match"
    queryParams = {"steamId": steamid, 
                   "heroId": heroid,
                   "include": "Player",
                   "playerType": "Single",
                   "gameMode": [2, 22],
                   "lobbyType": [2, 7],
                   "gameVersion": 79, 
                  "take": 100}
    x = requests.get(host, params = queryParams)
    x = x.json()
    features = {'kills': [], 'deaths': [], 'assists': [], 
                'lastHits': [], 'denies': [], 'gold': [], 'xp': [],
                'heroDamage': [], 'heroHealing': [], 'role': []}
    # operating on the game level
    for game in x['results']:
        # duration is in seconds
        duration = game['duration']
        features['kills'].append(game['players'][0]['numKills'])
        features['deaths'].append(game['players'][0]['numDeaths'])
        features['assists'].append(game['players'][0]['numAssists'])
        features['lastHits'].append(game['players'][0]['numLastHits'])
        features['denies'].append(game['players'][0]['numDenies'])
        features['gold'].append(game['players'][0]['goldPerMinute'] * 1.0 * duration/60)
        features['xp'].append(game['players'][0]['expPerMinute'] * 1.0 * duration/60)
        features['heroDamage'].append(game['players'][0]['heroDamage'])
        features['heroHealing'].append(game['players'][0]['heroHealing'])
        features['role'].append(game['players'][0]['role'])
    return {'kills': np.mean(features['kills']), 'deaths': np.mean(features['deaths']), 
            'assists': np.mean(features['assists']), 
            'lastHits': np.mean(features['lastHits']), 'denies': np.mean(features['denies']), 
            'gold': np.mean(features['gold']), 'xp': np.mean(features['xp']),
            'heroDamage': np.mean(features['heroDamage']), 'heroHealing': np.mean(features['heroHealing']), 
            'role': stats.mode(features['role'])[0][0]}

In [28]:
GetHeroHistory(54325937, 74)

<Response [200]>


{'assists': 10.714285714285714,
 'deaths': 5.0,
 'denies': 23.523809523809526,
 'gold': 23939.938095238093,
 'heroDamage': 28706.238095238095,
 'heroHealing': 0.0,
 'kills': 9.0952380952380949,
 'lastHits': 266.14285714285717,
 'role': 0,
 'xp': 26422.60396825397}

In [6]:
print('Loading invoker data')
if not os.path.exists('invokerMatchFeatureSetPlus.csv'):
    analyzeMatchesForFeatures(matchSet)
    invokerMatchFeatureSetPlus = pd.DataFrame(prototypeDataframe)
    invokerMatchFeatureSetPlus.to_csv('invokerMatchFeatureSetPlus.csv', index=False)
else:
    invokerMatchFeatureSetPlus = pd.read_csv('invokerMatchFeatureSetPlus.csv')

**Time to augment the invoker matches**

In [37]:
def invokerRowAugment(row):
    data = {}
    data['matchId'] = int(row['matchId'])
    # go over the four players on invoker side and five on other side
    for ix in range(5):
        heroInfo = GetHeroHistory(row['otherSideHero' + str(ix+1) + 'Steam'], row['otherSideHero' + str(ix+1)])
        data['OSHero' + str(ix+1) + 'hero'] = int(row['otherSideHero' + str(ix+1)])
        data['OSHero' + str(ix+1) + 'kills'] = heroInfo['kills']
        data['OSHero' + str(ix+1) + 'deaths'] = heroInfo['deaths']
        data['OSHero' + str(ix+1) + 'assists'] = heroInfo['assists']
        data['OSHero' + str(ix+1) + 'lastHits'] = heroInfo['lastHits']
        data['OSHero' + str(ix+1) + 'denies'] = heroInfo['denies']
        data['OSHero' + str(ix+1) + 'gold'] = heroInfo['gold']
        data['OSHero' + str(ix+1) + 'xp'] = heroInfo['xp']
        data['OSHero' + str(ix+1) + 'heroDamage'] = heroInfo['heroDamage']
        data['OSHero' + str(ix+1) + 'heroHealing'] = heroInfo['heroHealing']
        data['OSHero' + str(ix+1) + 'role'] = heroInfo['role']
        if ix != 4:
            heroInfo = GetHeroHistory(row['invokerSideHero' + str(ix+1) + 'Steam'], 
                                      row['invokerSideHero' + str(ix+1)])
            data['ISHero' + str(ix+1) + 'hero'] = int(row['invokerSideHero' + str(ix+1)])
            data['ISHero' + str(ix+1) + 'kills'] = heroInfo['kills']
            data['ISHero' + str(ix+1) + 'deaths'] = heroInfo['deaths']
            data['ISHero' + str(ix+1) + 'assists'] = heroInfo['assists']
            data['ISHero' + str(ix+1) + 'lastHits'] = heroInfo['lastHits']
            data['ISHero' + str(ix+1) + 'denies'] = heroInfo['denies']
            data['ISHero' + str(ix+1) + 'gold'] = heroInfo['gold']
            data['ISHero' + str(ix+1) + 'xp'] = heroInfo['xp']
            data['ISHero' + str(ix+1) + 'heroDamage'] = heroInfo['heroDamage']
            data['ISHero' + str(ix+1) + 'heroHealing'] = heroInfo['heroHealing']
            data['ISHero' + str(ix+1) + 'role'] = heroInfo['role']
    return pd.Series(data)

In [38]:
print('Loading supplemental data')
if not os.path.exists('supplementalInvokerMatchData.csv'):
    supplementalInvokerMatchData = invokerMatchFeatureSetPlus.apply(invokerRowAugment, axis=1)
    supplementalInvokerMatchData.to_csv('supplementalInvokerMatchData.csv', index=False)
else:
    supplementalInvokerMatchData = pd.read_csv('supplementalInvokerMatchData.csv')

,ISHero1assists,ISHero1deaths,ISHero1denies,ISHero1gold,ISHero1hero,ISHero1heroDamage,ISHero1heroHealing,ISHero1kills,ISHero1lastHits,ISHero1role,...,OSHero5denies,OSHero5gold,OSHero5hero,OSHero5heroDamage,OSHero5heroHealing,OSHero5kills,OSHero5lastHits,OSHero5role,OSHero5xp,matchId
0,20.142857,7.00,2.00,14722.040476,16.0,16415.0,0.00,6.571429,100.142857,1.0,...,16.50,10437.416667,13.0,12511.0,0.00,3.5,138.00,0.0,13053.316667,3.305849e+09
1,9.450000,4.65,17.95,24818.813333,48.0,24606.4,0.00,9.100000,349.450000,0.0,...,27.00,10129.600000,20.0,7111.0,0.00,4.0,89.00,0.0,10056.550000,3.312411e+09
2,14.000000,6.00,3.00,11157.066667,7.0,14788.0,0.00,4.000000,50.000000,0.0,...,1.00,10712.266667,5.0,12599.0,0.00,4.0,63.00,2.0,15382.666667,3.311329e+09
3,8.000000,5.50,2.50,16554.233333,9.0,9253.5,0.00,4.500000,174.000000,0.0,...,0.00,11110.400000,62.0,12089.0,0.00,3.0,27.00,1.0,14712.600000,3.290997e+09
4,8.700000,5.50,18.30,19258.345000,70.0,28067.8,712.05,14.400000,180.000000,0.0,...,9.45,25446.330833,25.0,27995.7,1678.65,12.9,354.85,0.0,26988.554167,3.303555e+09
